In [13]:
import os
from pyspark.sql import SparkSession
from dotenv import load_dotenv

load_dotenv()

spark = SparkSession.builder \
        .remote("sc://localhost:15002") \
        .appName("HybridDataPipelineSpark") \
        .getOrCreate()

In [15]:
jdbc_url = (
    f"jdbc:sqlserver://{os.getenv('AZURE_SQL_HOST')}:1433;"
    f"database={os.getenv('AZURE_SQL_DB')};"
    f"encrypt=true;"
    f"trustServerCertificate=false;"
)